# **Metin Ön İşleme**

In [1]:
# Temel Kütüphaneleri yükleme
import numpy as np
import pandas as pd
import re
import nltk
import spacy
import string


from nltk.corpus import twitter_samples
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem.snowball import SnowballStemmer
from nltk.stem import WordNetLemmatizer
from bs4 import BeautifulSoup


pd.options.display.max_columns=None
pd.options.display.max_rows=None
pd.options.display.max_colwidth=None

# Örnek bir veri seti yükleme
nltk.download('twitter_samples')

# Her biri 5000 tweet içeren Olumsuz ve Olumlu Tweetler dosyasını kullanacağız.
for name in twitter_samples.fileids():
    print(f' - {name}')

 - negative_tweets.json
 - positive_tweets.json
 - tweets.20150430-223406.json


[nltk_data] Downloading package twitter_samples to
[nltk_data]     C:\Users\emirr\AppData\Roaming\nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!


## Metin içerisindeki emojileri doğru şekilde kaldırmak ve değiştirmek için

In [2]:
pip install demoji

Note: you may need to restart the kernel to use updated packages.


In [3]:
import demoji

In [4]:
# Negatif tweet dosyasını yükleyin ve negatif için etiketi 0 olarak atayın
negative_tweets = twitter_samples.strings("negative_tweets.json")
df_neg = pd.DataFrame(negative_tweets, columns=['text'])
df_neg['label'] = 0

# Pozitif tweetler dosyasını yükleyin ve pozitifler için etiketi 1 olarak atayın
positive_tweets = twitter_samples.strings("positive_tweets.json")
df_pos = pd.DataFrame(positive_tweets, columns=['text'])
df_pos['label'] = 1

df = pd.concat([df_pos, df_neg]) # Her iki dosyayı da birleştir
df = df.sample(frac=1).reset_index(drop=True) # Negatif ve pozitif tweetleri karıştırmak için verileri karıştırın

In [5]:
print(f'Veriler: {df.shape[0]} satır ve {df.shape[1]} sutun dan oluşturulmuştur')

Veriler: 10000 satır ve 2 sutun dan oluşturulmuştur


In [6]:
df.head()  # ilk beş örnek

,text,label
0,@brighteyedmgc hoLY FUCK are u okay / text me if u need anythin !!! :-(,0
1,Add my KIK - lorm823 #kik #kikmeguys #blonde #rp #kikmenow #indiemusic #selfshot :( http://t.co/W07JltA7Fy,0
2,Having a bad headache right before a trip :(,0
3,@94tmhhes all these theories of people saying from the start that zayn will come back on 23rd july :((((((( its 24th today and im not ok,0
4,hrryok: the fact that harry still hold his mom’s hand :( :( :( http://t.co/hgE3nNSq4q,0


### Lower Casing (Küçük harfe çevirme)

In [7]:
df.text = df.text.str.lower()
df.head(5)

,text,label
0,@brighteyedmgc holy fuck are u okay / text me if u need anythin !!! :-(,0
1,add my kik - lorm823 #kik #kikmeguys #blonde #rp #kikmenow #indiemusic #selfshot :( http://t.co/w07jlta7fy,0
2,having a bad headache right before a trip :(,0
3,@94tmhhes all these theories of people saying from the start that zayn will come back on 23rd july :((((((( its 24th today and im not ok,0
4,hrryok: the fact that harry still hold his mom’s hand :( :( :( http://t.co/hge3nnsq4q,0


## Silme işlemleri

In [8]:
df.text = df.text.str.replace(r'https?://\S+|www\.\S+', '', regex=True)
df.head()

,text,label
0,@brighteyedmgc holy fuck are u okay / text me if u need anythin !!! :-(,0
1,add my kik - lorm823 #kik #kikmeguys #blonde #rp #kikmenow #indiemusic #selfshot :(,0
2,having a bad headache right before a trip :(,0
3,@94tmhhes all these theories of people saying from the start that zayn will come back on 23rd july :((((((( its 24th today and im not ok,0
4,hrryok: the fact that harry still hold his mom’s hand :( :( :(,0


In [9]:
text = 'I have being trying to contact xyz via email to xyz@abc.co.in but there is no response.'
re.sub(r'\S+@\S+', '', text)

'I have being trying to contact xyz via email to  but there is no response.'

In [10]:
text = "Today is 22/12/2020 and after two days on 24-12-2020 our vacation starts until 25th.09.2021"

# 1. Remove date formats like: dd/mm/yy(yy), dd-mm-yy(yy), dd(st|nd|rd).mm/yy(yy)
re.sub(r'\d{1,2}(st|nd|rd|th)?[-./]\d{1,2}[-./]\d{2,4}', '', text)

'Today is  and after two days on  our vacation starts until '

In [11]:
text = """
<title>Below is a dummy html code.</title>
<body>
    <p>All the html opening and closing brackets should be remove.</p>
    <a href="https://www.abc.com">Company Site</a>
</body>
"""
# regex kullanarak
pattern = re.compile('<.*?>')
pattern.sub('', text)

'\nBelow is a dummy html code.\n\n    All the html opening and closing brackets should be remove.\n    Company Site\n\n'

In [12]:
text = """
<title>Below is a dummy html code.</title>
<body>
    <p>All the html opening and closing brackets should be remove.</p>
    <a href="https://www.abc.com">Company Site</a>
</body>
"""
# Beautiful Soup kullanarak
def remove_html(text):
    clean_text = BeautifulSoup(text).get_text()
    return clean_text

remove_html(text)

'\nBelow is a dummy html code.\n\nAll the html opening and closing brackets should be remove.\nCompany Site\n\n'

#### emoji silme işlemi

In [13]:
# Reference: https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b

def remove_emoji(text):
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002500-\U00002BEF"  # chinese char
                               u"\U00002702-\U000027B0"
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               u"\U0001f926-\U0001f937"
                               u"\U00010000-\U0010ffff"
                               u"\u2640-\u2642"
                               u"\u2600-\u2B55"
                               u"\u200d"
                               u"\u23cf"
                               u"\u23e9"
                               u"\u231a"
                               u"\ufe0f"  # dingbats
                               u"\u3030"
                               "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

##### örnek

In [14]:
text = "game is on 🔥🔥. Hilarious😂"
remove_emoji(text)

'game is on . Hilarious'

In [15]:
# veri setinden emojileri silme
df.text = df.text.apply(lambda x: remove_emoji(x))

#### karakterler kullanılarak yapılan emojiler

In [16]:
EMOTICONS = {
    u":‑\)":"Happy face or smiley",
    u":\)":"Happy face or smiley",
    u":-\]":"Happy face or smiley",
    u":\]":"Happy face or smiley",
    u":-3":"Happy face smiley",
    u":3":"Happy face smiley",
    u":->":"Happy face smiley",
    u":>":"Happy face smiley",
    u"8-\)":"Happy face smiley",
    u":o\)":"Happy face smiley",
    u":-\}":"Happy face smiley",
    u":\}":"Happy face smiley",
    u":-\)":"Happy face smiley",
    u":c\)":"Happy face smiley",
    u":\^\)":"Happy face smiley",
    u"=\]":"Happy face smiley",
    u"=\)":"Happy face smiley",
    u":‑D":"Laughing, big grin or laugh with glasses",
    u":D":"Laughing, big grin or laugh with glasses",
    u"8‑D":"Laughing, big grin or laugh with glasses",
    u"8D":"Laughing, big grin or laugh with glasses",
    u"X‑D":"Laughing, big grin or laugh with glasses",
    u"XD":"Laughing, big grin or laugh with glasses",
    u"=D":"Laughing, big grin or laugh with glasses",
    u"=3":"Laughing, big grin or laugh with glasses",
    u"B\^D":"Laughing, big grin or laugh with glasses",
    u":-\)\)":"Very happy",
    u":‑\(":"Frown, sad, andry or pouting",
    u":-\(":"Frown, sad, andry or pouting",
    u":\(":"Frown, sad, andry or pouting",
    u":‑c":"Frown, sad, andry or pouting",
    u":c":"Frown, sad, andry or pouting",
    u":‑<":"Frown, sad, andry or pouting",
    u":<":"Frown, sad, andry or pouting",
    u":‑\[":"Frown, sad, andry or pouting",
    u":\[":"Frown, sad, andry or pouting",
    u":-\|\|":"Frown, sad, andry or pouting",
    u">:\[":"Frown, sad, andry or pouting",
    u":\{":"Frown, sad, andry or pouting",
    u":@":"Frown, sad, andry or pouting",
    u">:\(":"Frown, sad, andry or pouting",
    u":'‑\(":"Crying",
    u":'\(":"Crying",
    u":'‑\)":"Tears of happiness",
    u":'\)":"Tears of happiness",
    u"D‑':":"Horror",
    u"D:<":"Disgust",
    u"D:":"Sadness",
    u"D8":"Great dismay",
    u"D;":"Great dismay",
    u"D=":"Great dismay",
    u"DX":"Great dismay",
    u":‑O":"Surprise",
    u":O":"Surprise",
    u":‑o":"Surprise",
    u":o":"Surprise",
    u":-0":"Shock",
    u"8‑0":"Yawn",
    u">:O":"Yawn",
    u":-\*":"Kiss",
    u":\*":"Kiss",
    u":X":"Kiss",
    u";‑\)":"Wink or smirk",
    u";\)":"Wink or smirk",
    u"\*-\)":"Wink or smirk",
    u"\*\)":"Wink or smirk",
    u";‑\]":"Wink or smirk",
    u";\]":"Wink or smirk",
    u";\^\)":"Wink or smirk",
    u":‑,":"Wink or smirk",
    u";D":"Wink or smirk",
    u":‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"X‑P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"XP":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":Þ":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"d:":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"=p":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u">:P":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u":‑/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":-[.]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u">:/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=/":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=[(\\\)]":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u"=L":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":S":"Skeptical, annoyed, undecided, uneasy or hesitant",
    u":‑\|":"Straight face",
    u":\|":"Straight face",
    u":$":"Embarrassed or blushing",
    u":‑x":"Sealed lips or wearing braces or tongue-tied",
    u":x":"Sealed lips or wearing braces or tongue-tied",
    u":‑#":"Sealed lips or wearing braces or tongue-tied",
    u":#":"Sealed lips or wearing braces or tongue-tied",
    u":‑&":"Sealed lips or wearing braces or tongue-tied",
    u":&":"Sealed lips or wearing braces or tongue-tied",
    u"O:‑\)":"Angel, saint or innocent",
    u"O:\)":"Angel, saint or innocent",
    u"0:‑3":"Angel, saint or innocent",
    u"0:3":"Angel, saint or innocent",
    u"0:‑\)":"Angel, saint or innocent",
    u"0:\)":"Angel, saint or innocent",
    u":‑b":"Tongue sticking out, cheeky, playful or blowing a raspberry",
    u"0;\^\)":"Angel, saint or innocent",
    u">:‑\)":"Evil or devilish",
    u">:\)":"Evil or devilish",
    u"\}:‑\)":"Evil or devilish",
    u"\}:\)":"Evil or devilish",
    u"3:‑\)":"Evil or devilish",
    u"3:\)":"Evil or devilish",
    u">;\)":"Evil or devilish",
    u"\|;‑\)":"Cool",
    u"\|‑O":"Bored",
    u":‑J":"Tongue-in-cheek",
    u"#‑\)":"Party all night",
    u"%‑\)":"Drunk or confused",
    u"%\)":"Drunk or confused",
    u":-###..":"Being sick",
    u":###..":"Being sick",
    u"<:‑\|":"Dump",
    u"\(>_<\)":"Troubled",
    u"\(>_<\)>":"Troubled",
    u"\(';'\)":"Baby",
    u"\(\^\^>``":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(\^_\^;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(~_~;\) \(・\.・;\)":"Nervous or Embarrassed or Troubled or Shy or Sweat drop",
    u"\(-_-\)zzz":"Sleeping",
    u"\(\^_-\)":"Wink",
    u"\(\(\+_\+\)\)":"Confused",
    u"\(\+o\+\)":"Confused",
    u"\(o\|o\)":"Ultraman",
    u"\^_\^":"Joyful",
    u"\(\^_\^\)/":"Joyful",
    u"\(\^O\^\)／":"Joyful",
    u"\(\^o\^\)／":"Joyful",
    u"\(__\)":"Kowtow as a sign of respect, or dogeza for apology",
    u"_\(\._\.\)_":"Kowtow as a sign of respect, or dogeza for apology",
    u"<\(_ _\)>":"Kowtow as a sign of respect, or dogeza for apology",
    u"<m\(__\)m>":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(__\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"m\(_ _\)m":"Kowtow as a sign of respect, or dogeza for apology",
    u"\('_'\)":"Sad or Crying",
    u"\(/_;\)":"Sad or Crying",
    u"\(T_T\) \(;_;\)":"Sad or Crying",
    u"\(;_;":"Sad of Crying",
    u"\(;_:\)":"Sad or Crying",
    u"\(;O;\)":"Sad or Crying",
    u"\(:_;\)":"Sad or Crying",
    u"\(ToT\)":"Sad or Crying",
    u";_;":"Sad or Crying",
    u";-;":"Sad or Crying",
    u";n;":"Sad or Crying",
    u";;":"Sad or Crying",
    u"Q\.Q":"Sad or Crying",
    u"T\.T":"Sad or Crying",
    u"QQ":"Sad or Crying",
    u"Q_Q":"Sad or Crying",
    u"\(-\.-\)":"Shame",
    u"\(-_-\)":"Shame",
    u"\(一一\)":"Shame",
    u"\(；一_一\)":"Shame",
    u"\(=_=\)":"Tired",
    u"\(=\^\·\^=\)":"cat",
    u"\(=\^\·\·\^=\)":"cat",
    u"=_\^=	":"cat",
    u"\(\.\.\)":"Looking down",
    u"\(\._\.\)":"Looking down",
    u"\^m\^":"Giggling with hand covering mouth",
    u"\(\・\・?":"Confusion",
    u"\(?_?\)":"Confusion",
    u">\^_\^<":"Normal Laugh",
    u"<\^!\^>":"Normal Laugh",
    u"\^/\^":"Normal Laugh",
    u"\（\*\^_\^\*）" :"Normal Laugh",
    u"\(\^<\^\) \(\^\.\^\)":"Normal Laugh",
    u"\(^\^\)":"Normal Laugh",
    u"\(\^\.\^\)":"Normal Laugh",
    u"\(\^_\^\.\)":"Normal Laugh",
    u"\(\^_\^\)":"Normal Laugh",
    u"\(\^\^\)":"Normal Laugh",
    u"\(\^J\^\)":"Normal Laugh",
    u"\(\*\^\.\^\*\)":"Normal Laugh",
    u"\(\^—\^\）":"Normal Laugh",
    u"\(#\^\.\^#\)":"Normal Laugh",
    u"\（\^—\^\）":"Waving",
    u"\(;_;\)/~~~":"Waving",
    u"\(\^\.\^\)/~~~":"Waving",
    u"\(-_-\)/~~~ \($\·\·\)/~~~":"Waving",
    u"\(T_T\)/~~~":"Waving",
    u"\(ToT\)/~~~":"Waving",
    u"\(\*\^0\^\*\)":"Excited",
    u"\(\*_\*\)":"Amazed",
    u"\(\*_\*;":"Amazed",
    u"\(\+_\+\) \(@_@\)":"Amazed",
    u"\(\*\^\^\)v":"Laughing,Cheerful",
    u"\(\^_\^\)v":"Laughing,Cheerful",
    u"\(\(d[-_-]b\)\)":"Headphones,Listening to music",
    u'\(-"-\)':"Worried",
    u"\(ーー;\)":"Worried",
    u"\(\^0_0\^\)":"Eyeglasses",
    u"\(\＾ｖ\＾\)":"Happy",
    u"\(\＾ｕ\＾\)":"Happy",
    u"\(\^\)o\(\^\)":"Happy",
    u"\(\^O\^\)":"Happy",
    u"\(\^o\^\)":"Happy",
    u"\)\^o\^\(":"Happy",
    u":O o_O":"Surprised",
    u"o_0":"Surprised",
    u"o\.O":"Surpised",
    u"\(o\.o\)":"Surprised",
    u"oO":"Surprised",
    u"\(\*￣m￣\)":"Dissatisfied",
    u"\(‘A`\)":"Snubbed or Deflated"
}

In [17]:
def remove_emoticons(text):
    emoticons_pattern = re.compile(u'(' + u'|'.join(emo for emo in EMOTICONS) + u')')
    return emoticons_pattern.sub(r'', text)

In [18]:
remove_emoticons("Hello :->")

'Hello '

In [19]:
# veri setinden emojileri silme
df.text = df.text.apply(lambda x: remove_emoticons(x))

### emojileri metne dönüştürme

In [20]:
demoji.download_codes()

C:\Users\emirr\AppData\Local\Temp\ipykernel_11732\2299328559.py:1: FutureWarning: The demoji.download_codes attribute is deprecated and will be removed from demoji in a future version. It is an unused attribute as emoji codes are now distributed directly with the demoji package.
  demoji.download_codes()


In [21]:
def emoji_to_words(text):
    return demoji.replace_with_desc(text, sep="__")

In [22]:
text = "game is on 🔥 🚣🏼"
emoji_to_words(text)

'game is on __fire__ __person rowing boat: medium-light skin tone__'

In [23]:
def emoticons_to_words(text):
    for emot in EMOTICONS:
        text = re.sub(u'('+emot+')', "_".join(EMOTICONS[emot].replace(",","").replace(":","").split()), text)
    return text

In [24]:
text = "Hey there!! :-)"
emoticons_to_words(text)

'Hey there!! Happy_face_smiley'

In [25]:
def remove_tags_mentions(text):
    pattern = re.compile(r'(@\S+|#\S+)')
    return pattern.sub('', text)

In [26]:
text = "live @flippinginja on #younow - jonah and jareddddd"
remove_tags_mentions(text)

'live  on  - jonah and jareddddd'

In [27]:
# veri setinden hastagları silme
df.text = df.text.apply(lambda x: remove_tags_mentions(x))

#### Noktalama işaretleri kaldırma

In [28]:
PUNCTUATIONS = string.punctuation

def remove_punctuation(text):
    return text.translate(str.maketrans('', '', PUNCTUATIONS))

In [29]:
df.text = df["text"].apply(lambda text: remove_punctuation(text))

#### Stopwords kaldırma

In [30]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\emirr\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
STOPWORDS = set(stopwords.words('english'))

def remove_stopwords(text):
    return ' '.join([word for word in text.split() if word not in STOPWORDS])

In [32]:
df.text = df.text.apply(lambda text: remove_stopwords(text))
df.head()

,text,label
0,holy fuck u okay text u need anythin,0
1,add kik lorm823,0
2,bad headache right trip,0
3,theories people saying start zayn come back 23rd july 24th today im ok,0
4,hrryok fact harry still hold mom’s hand,0


#### fazla boşlukları silme

In [33]:
def remove_whitespaces(text):
    return " ".join(text.split())

In [34]:
text = "  Whitespaces in the beginning are removed  \t as well \n  as in between  the text   "

clean_text = " ".join(text.split())
clean_text

'Whitespaces in the beginning are removed as well as in between the text'

In [35]:
df.text = df.text.apply(lambda x: remove_whitespaces(x))

### Stemming





In [36]:
from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

def stem_words(text):
    return ' '.join([stemmer.stem(word) for word in text.split()])

In [37]:
df['text_stemmed'] = df.text.apply(lambda text: stem_words(text))
df[['text', 'text_stemmed']].head()

,text,text_stemmed
0,holy fuck u okay text u need anythin,holi fuck u okay text u need anythin
1,add kik lorm823,add kik lorm823
2,bad headache right trip,bad headach right trip
3,theories people saying start zayn come back 23rd july 24th today im ok,theori peopl say start zayn come back 23rd juli 24th today im ok
4,hrryok fact harry still hold mom’s hand,hrryok fact harri still hold mom’ hand


### Lemmatization


In [38]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\emirr\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
lemmatizer = WordNetLemmatizer()

def text_lemmatize(text):
    return ' '.join([lemmatizer.lemmatize(word) for word in text.split()])

In [40]:
df['text_lemmatized'] = df.text.apply(lambda text: text_lemmatize(text))
df[['text', 'text_stemmed', 'text_lemmatized']].head()

,text,text_stemmed,text_lemmatized
0,holy fuck u okay text u need anythin,holi fuck u okay text u need anythin,holy fuck u okay text u need anythin
1,add kik lorm823,add kik lorm823,add kik lorm823
2,bad headache right trip,bad headach right trip,bad headache right trip
3,theories people saying start zayn come back 23rd july 24th today im ok,theori peopl say start zayn come back 23rd juli 24th today im ok,theory people saying start zayn come back 23rd july 24th today im ok
4,hrryok fact harry still hold mom’s hand,hrryok fact harri still hold mom’ hand,hrryok fact harry still hold mom’s hand


### Yazım hatası düzeltme


In [41]:
pip install pyspellchecker

Note: you may need to restart the kernel to use updated packages.


In [42]:
from spellchecker import SpellChecker

In [43]:
spell = SpellChecker()

def correct_spelling(text):
    correct_text = []
    misspelled_words = spell.unknown(text.split())
    for word in text.split():
        if word in misspelled_words:
            correct_text.append(spell.correction(word))
        else:
            correct_text.append(word)
    return " ".join(correct_text)

In [44]:
text = "Hi, hwo are you doin? I'm good thnks for asking"
correct_spelling(text)

"Hi, how are you doing I'm good thanks for asking"

In [45]:
text = "hw are you doin? I'm god thnks"
correct_spelling(text)

"he are you doing I'm god thanks"